In [84]:
from unstructured.partition.pdf import partition_pdf

## PDF

In [86]:
output_path = "images"
# file_path = "Attention.pdf"
file_path="LoRA_and_QLora.pdf"

chunks = partition_pdf(
			filename=file_path,
			infer_table_structure=True,  # extract tables
			strategy="hi_res",  # mandatory to infer tables
			extract_image_block_types=[
				"Image"
			],  # Add 'Table' to list to extract image of tables
			# image_output_dir_path=output_path,  # if None, images and tables will saved in base64
			extract_image_block_to_payload=True,  # if true, will extract base64 for API usage
			chunking_strategy="by_title",  # or 'basic', by_page - api
			max_characters=10000,  # defaults to 500
			combine_text_under_n_chars=2000,  # defaults to 0
			new_after_n_chars=6000,
			# extract_images_in_pdf=True,          # deprecated
		)

RuntimeError: Failed to import transformers.models.detr.image_processing_detr because of the following error (look up to see its traceback):
Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [7]:
chunk.metadata

### DOCX

In [116]:
import docx

from unstructured.partition.docx import partition_docx

from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.oxml import OxmlElement

from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.oxml import OxmlElement

# Create the document
document = Document()
document.add_paragraph("Important Analysis", style="Heading 1")
document.add_paragraph("Here is my first thought.", style="Body Text")
document.add_paragraph("Here is my second thought.", style="Normal")
document.add_page_break()
document.add_paragraph("my name sanket")



# elements = partition_docx(filename="mydoc.docx")

# with open("mydoc.docx", "rb") as f:
elements = partition_docx(filename="LoRA_and_QLora.docx",
                        strategy="hi_res",
                        infer_table_structure=True, 
                        include_page_breaks=True,
                        extract_image_block_types=["Image"], 
                        extract_image_block_to_payload=True,
                        chunking_strategy="by_title",
                        max_characters=10000,
                        combine_text_under_n_chars=2000,  # defaults to 0
                        new_after_n_chars=6000)

In [117]:
# for id, chunk in enumerate(elements):
#     chunk_dict = chunk.to_dict()
#     print(chunk_dict)

In [118]:
# chunk_dict['metadata']

# llama-3.3-70b-versatile

## doc

In [37]:
from unstructured.partition.email import partition_email

filename = "How Meta Plans To Crank the Dopamine Machine With Infinite AI-Generated Content _ Alberto Romero.eml"
elements = partition_email(filename=filename, 
                        chunking_strategy="basic",
                           process_attachments=True, 
                           include_orig_elements=True)

In [71]:
from unstructured.partition.html import partition_html

elements = partition_html(filename="sample_html.html",
                        strategy="hi_res",
                        infer_table_structure=True, 
                        include_page_breaks=True,
                        extract_image_block_types=["Image"], 
                        extract_image_block_to_payload=True,
                        chunking_strategy="basic",
                        max_characters=10000,
                        combine_text_under_n_chars=2000,  # defaults to 0
                        new_after_n_chars=6000,
                         include_orig_elements=True)

In [77]:
for id, chunk in enumerate(elements):
    chunk_dict = chunk.to_dict()
    print(chunk_dict)

{'type': 'Title', 'element_id': '968dd6da53ddecf37784520551aa14f9', 'metadata': {'filename': 'sample_html.html', 'filetype': 'text/html', 'page_number': 1}, 'text': 'Sample HTML Document'}
{'type': 'NarrativeText', 'element_id': 'c22865cae4b28817ec1a6681d114e883', 'metadata': {'filename': 'sample_html.html', 'filetype': 'text/html', 'page_number': 1}, 'text': 'This is a sample HTML document demonstrating basic structure, including a heading and a paragraph.'}


In [79]:
chunk.metadata

ElementMetadata(coordinates=None, data_source=None, filename='sample_html.html', file_directory=None, date=None, filetype='text/html', attached_to_filename=None, page_number=1, page_name=None, url=None, sent_from=None, sent_to=None, subject=None, header_footer_type=None, text_as_html=None, regex_metadata={})

In [80]:
list1=[]
for id, chunk in enumerate(elements):
    chunk_dict = chunk.to_dict()
    elements = chunk.metadata.orig_elements
    chunk_images = [el.to_dict() for el in elements if "Image" in str(type(el))]
    # print(chunk_images)
    item = {}
    if "metadata" not in item.keys():
        item['metadata']={}
    # item["metadata"]["chunk_id"] = file_Path + f"_{id}"
    # chunk_ids.append(item["metadata"]["chunk_id"])
    item["metadata"]["filename"] = chunk_dict["metadata"]["filename"]
    item["metadata"]["element_id"] = chunk_dict["element_id"]
    # item["metadata"]['domain']=domain
    if "Table" in str(elements):
        print("Table Detected")
        item["page_content"] = (
            chunk_dict["text"]
            + "Table in Html format :"
            + chunk_dict["metadata"]["text_as_html"]
        )
    else:
        item["page_content"] = chunk_dict["text"]
    if "page_number" in chunk_dict["metadata"].keys():
        item["metadata"]["page_number"] = str(chunk_dict["metadata"]["page_number"])
    else:
        item["metadata"]["page_number"] = "NA"
    if chunk_images:
        print("Image Detected")
        item["metadata"]["images_info"] = str(
            [
                {
                    "image_associated_text": i["text"],
                    "image_base64": i["metadata"]["image_base64"],
                    "images_link": "",
                }
                for i in chunk_images
            ]
        )
    # item["chunk_vector"] = generate_embeddings_openai(item["chunk"], client)
    list1.append(item)


AttributeError: 'ElementMetadata' object has no attribute 'orig_elements'

In [82]:
# list1

In [1]:
from langchain_elasticsearch import ElasticsearchStore
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
{
  "id": "jY_RIZQBuIghXv4v-UV9",
  "name": "key1",
  "api_key": "mhfCpHJeSWedes0VhRKwwg",
  "encoded": "allfUklaUUJ1SWdoWHY0di1VVjk6bWhmQ3BISmVTV2VkZXMwVmhSS3d3Zw==",
  "beats_logstash_format": "jY_RIZQBuIghXv4v-UV9:mhfCpHJeSWedes0VhRKwwg"
}

In [3]:
embeddings_model = HuggingFaceEmbeddings(
    model_name ="jinaai/jina-embeddings-v2-small-en", # switch to en/zh for English or Chinese
    # model_kwargs={"max_seq_length":1024}
	)

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.3.intermediate.den

In [7]:
elastic_vector_search = ElasticsearchStore(
# es_cloud_id="35b3d239e2fe494391812696da8f9f25:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJGU3MjE0ZjU1MjczNjRjOTFiOWU1MzFhNmE3YjVjM2Q1JDM3M2ExNDdkMDRmMTQzMzJiYTIwMDA0YjI5ZTZkODVh",
es_url="https://8cbf006f88574954ae562356e9fb5249.us-central1.gcp.cloud.es.io:443",
index_name="metamind",
embedding=embeddings_model,
es_api_key="dzdMVklaUUJfUVFMeTF0cU10Q1M6MkVaUmN2UDBRX21rSXJPVWtYMmI0Zw==",
# es_user="elastic",
# es_password="changeme",
)

In [13]:
results=elastic_vector_search.search(query="what is qlora and lora?",search_type="mmr",
    k=2)

In [14]:
# results

[Document(metadata={'chunk_id': 'hr_LoRA_and_QLora_4.docx_1', 'filename': 'hr_LoRA_and_QLora_4.docx', 'element_id': 'c4f4cf60a4836bd8905d99952ea942a1', 'domain': 'hr', 'page_number': '2'}, page_content='Low-Rank Adaptation (LoRA)\n\nLow-Rank Adaptation provides the modular approach towards to fine-tuning a model for domains specific tasks and provides the capability of transfer learning.\n\nLoRA technique can be implemented with fewer resources and are memory efficient.\n\nIn the following picture you can see the dimension/rank decomposition, that reduces the memory footprint considerably.\n\nFigure 2\n\nLoRA network for Training\n\nFigure\n\nSteps:\n\nLoRA can be implemented as an adapter designed to enhance and expand the existing neural network layers. It introduces an additional layer of trainable parameters (weights) while maintaining the original parameters in a frozen state. These trainable parameters possess a substantially reduced rank (dimension) compared to the dimensions of

In [11]:
results = elastic_vector_search.similarity_search_with_score(
    query="what is qlora and lora?",
    k=2
)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* [SIM=0.725636] Low-Rank Adaptation (LoRA)

Low-Rank Adaptation provides the modular approach towards to fine-tuning a model for domains specific tasks and provides the capability of transfer learning.

LoRA technique can be implemented with fewer resources and are memory efficient.

In the following picture you can see the dimension/rank decomposition, that reduces the memory footprint considerably.

Figure 2

LoRA network for Training

Figure

Steps:

LoRA can be implemented as an adapter designed to enhance and expand the existing neural network layers. It introduces an additional layer of trainable parameters (weights) while maintaining the original parameters in a frozen state. These trainable parameters possess a substantially reduced rank (dimension) compared to the dimensions of the original network. This is the mechanism through which LoRa simplifies and speeds up adapting the original models for domain-specific tasks. Now, let’s take a closer look at the components within th